In [1]:
# August 2023
# pcd (pointcloud data) files
# PCD file format is binary compressed PointCloudData

In [2]:
# 1) Open up Cloud Compare softw and  and convert '.laz' to '.las' (or save as .pcd)
# 2) downsample (if you like) in Cloud Compare 'Edit+Octree+Resample'
# 3) reading cloud.pcd like down bellow.
# 4) convert it to standard cubic n*n*n like bellow
# 5) save in '.xyz' or visualize is like bellow

In [3]:
import open3d as o3d
from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
file_name = Path("lidar data/forest-sence-sample.pcd")
pcd = o3d.io.read_point_cloud(str(file_name))
pcd

PointCloud with 236886 points.

In [5]:
# voxelization
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,
                                                            voxel_size=0.05)

In [6]:
# Visualization
o3d.visualization.draw_geometries([voxel_grid])

In [7]:
import numpy as np
np.asarray(pcd.points).shape

(236886, 3)

In [8]:
voxel_grid.get_axis_aligned_bounding_box()

AxisAlignedBoundingBox: min: (-16.012, -16.038, -5.034), max: (16.038, 16.012, 33.966)

In [9]:
voxel_grid.get_center()

array([ 0.84989998, -0.12012058, 20.54368776])

# grab indeces from pointcloud file

In [10]:
# data at these indeces are 1 (there is a point at these locations with
# color) otherwise there is no points. Simply put, if there is no indeces 
# of your interest, it means there is no point at that index.
voxel_grid.get_voxels()[0:10]

[Voxel with grid_index: (306, 435, 615), color: (0, 0, 0),
 Voxel with grid_index: (318, 429, 621), color: (0, 0, 0),
 Voxel with grid_index: (309, 427, 618), color: (0, 0, 0),
 Voxel with grid_index: (221, 353, 400), color: (0, 0, 0),
 Voxel with grid_index: (522, 559, 135), color: (0, 0, 0),
 Voxel with grid_index: (319, 423, 619), color: (0, 0, 0),
 Voxel with grid_index: (405, 145, 584), color: (0, 0, 0),
 Voxel with grid_index: (318, 429, 614), color: (0, 0, 0),
 Voxel with grid_index: (219, 615, 563), color: (0, 0, 0),
 Voxel with grid_index: (312, 429, 615), color: (0, 0, 0)]

In [11]:
# just taking voxels' indeces, not color info
np.stack(list(vx.grid_index for vx in voxel_grid.get_voxels()))

array([[306, 435, 615],
       [318, 429, 621],
       [309, 427, 618],
       ...,
       [383, 137, 575],
       [582, 271, 600],
       [ 78, 515, 559]], dtype=int32)

In [13]:
# all
voxels = voxel_grid.get_voxels()  # returns list of voxels
indices = np.stack(list(vx.grid_index for vx in voxels))
colors = np.stack(list(vx.color for vx in voxels))

# Standardize the indeces 

In [ ]:
# We need sth that works with max and min of indeces and put them right into a cubic shape:
# first we need a way to convert indeces standard. e.g. if we want a cube of size=256, we
# should have 0 and 255 as our min and max index:
coef_cubicize = (indices.max(axis=0)-indices.min(axis=0))/255
coef_cubicize

In [ ]:
# to put indeces from 0--255
cube_voxels = (indices-indices.min(axis=0))//coef_cubicize
cube_voxels = cube_voxels.astype(int)
cube_voxels.max(axis=0)

In [ ]:
# Sample: 
from utils import cube_voxel
# cube_voxel function returns indeces which have the value==1
# also cram/stretch data into a cube_size*cube_size*cube_size cube.
cube_voxel(indices, cube_size=32)

In [ ]:
cube_voxel(indices, 32).max(axis=0)

#  Cubic data creation with indeces info

In [ ]:
from utils import build_cube

In [ ]:
cube_size = 256
std_indices = cube_voxel(indices, cube_size)
occupancy_cube_vox = build_cube(indices=std_indices, cube_size=cube_size)

In [ ]:
# now occupancy_cube_vox represent the sample data in a standard cubic shape
occupancy_cube_vox.shape

In [ ]:
# To save the data

In [ ]:
from utils import save_xyz, save_mat
save_xyz(occupancy_cube_vox, 'output.xyz')

In [ ]:
save_mat(occupancy_cube_vox, 'output.mat')

In [ ]:
occupancy_cube_vox.shape

# Next

In [ ]:
# - the occupancy_cube_vox.xyz can be shown in meshlab or Cloudcompare to see the look of the output file.
# - the occupancy_cube_vox.mat data that is built now can be imported in MATLAB to convert it into a .mat octree
# type.